### 모듈, 패키지 import

In [288]:
import os
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import urllib.parse as urlparse
import requests
from matplotlib import rc
rc('font', family = "Arial Unicode MS")
# from urllib3 import request, urlopen
import urllib.request

### 데이터 로드

In [289]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

# Train + Test 합치기

In [290]:
train = train.append(test)
train

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,중식메뉴,현본사소속재택근무자수
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",0.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",0.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",0.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",0.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",0.0
5,417,2601,143,383,550.0,"참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",목,2016-02-11,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,1045.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 시래기국 훈제오리구이 도토리묵무침 쌈무/양...",0.0
6,93,2601,156,389,598.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치콩나물국 미니함박 어묵볶음 물파래무침 ...",금,2016-02-12,모닝롤/야채샌드 우유/두유/주스 계란후라이 고구마죽/쌀밥 (쌀:국내산) 봄동된...,909.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 꽃게탕 돈육굴소스볶음 옥수수전 유채나물 ...",0.0
7,482,2601,204,87,672.0,"쌀밥/잡곡밥 (쌀:국내산) 홍합미역국 등갈비김치찜 (돼지고기,김치:국내산) 임연수...",월,2016-02-15,모닝롤/치즈프레즐 우유/두유/주스 계란후라이 잣죽/쌀밥 (쌀:국내산) 민물새우...,1268.0,쌀밥/잡곡밥 (쌀:국내산) 시금치국 닭감자조림 (닭고기:국내산) 연두부*양념장 ...,0.0
8,526,2601,236,72,523.0,쌀밥/잡곡밥 (쌀:국내산) 된장찌개 쇠불고기 (쇠고기:호주산) 해파리겨자채 봄동...,화,2016-02-16,모닝롤/마늘빵 우유/두유/주스 계란후라이 단호박죽/쌀밥 (쌀:국내산) 어묵국 ...,1014.0,쌀밥/잡곡밥 (쌀:국내산) 쇠고기무국 (쇠고기:호주산) 탕수어 (동태:러시아산) 오...,0.0
9,23,2601,250,78,588.0,볶음밥*자장소스 (쌀:국내산) 맑은국 새우또띠아 쨔샤이무침 요플레 포기김치 ...,수,2016-02-17,모닝롤/참치샌드 우유/두유/주스 계란후라이 흑임자죽/쌀밥 (쌀:국내산) 북어계...,916.0,쌀밥/잡곡밥 (쌀:국내산) 냉이된장국 쇠고기장조림 (쇠고기:호주산) 통도라지구이 ...,0.0


### 공휴일 데이터 받기

In [291]:
# 정부 API 이용: 토, 일을 제외한 국경일 및 공휴일 추가
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
operation = "getRestDeInfo"
mykey = 'ac%2FSOW4KzOFKdu0z01iEVWGZd4TBl1MyiR04%2FfYmADthCjJBEyL73pewbapUk94Gm1%2FqtzMCban3C%2BpqU8c7ew%3D%3D'

date = []
datename = []
for year in range(2016, 2022):
    year = str(year)
    
    for month in range(1, 13):
        if month < 10:
            month = "0" + str(month)
        else:
            month = str(month)
            
        params = {'solYear' : year, 'solMonth' : month}
        rq_query = url +'/' + operation + '?' + urlparse.urlencode(params) + "&serviceKey=" + mykey    
        response = requests.get(rq_query) 
        dom = BeautifulSoup(response.content, "html.parser")
        
        items = dom.find_all("item")
        for item in items:
            date.append(item.locdate.string)
            datename.append(item.datename.string)

holiday_df= pd.DataFrame({"date": date, "datename": datename})

In [292]:
holiday_df.head()

,date,datename
0,20160101,신정
1,20160207,설날
2,20160208,설날
3,20160209,설날
4,20160210,대체공휴일


In [293]:
# 휴일 전날
holiday_before_df = holiday_df.copy()
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strptime(x,'%Y%m%d'))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : x - relativedelta(days=1))
holiday_before_df['date'] = holiday_before_df['date'].apply(lambda x : datetime.strftime(x, '%Y%m%d'))
holiday_before_df['datename'] = holiday_df['datename'].apply(lambda x : 1)
holiday_before_df.columns = ['일자', '휴일전날']
holiday_before_df.head()

,일자,휴일전날
0,20151231,1
1,20160206,1
2,20160207,1
3,20160208,1
4,20160209,1


In [294]:
# 휴일전날 merge
train_df = pd.merge(train, holiday_before_df, left_on='일자', right_on='일자', how='left')
before_friday = train_df[train_df['요일']=='금'].index
train_df['휴일전날'][before_friday] = 1

In [295]:
# 평일은 0 처리
train_df['휴일전날'] = train_df['휴일전날'].fillna(0)

In [296]:
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,중식메뉴,현본사소속재택근무자수,휴일전날
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",0.0,0.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",0.0,0.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",0.0,0.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",0.0,0.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",0.0,1.0


### 강수량 데이터 추가

In [297]:
# 기상청에서 다운로드한 진주시 강수량 데이터 활용
rain_df = pd.DataFrame()
for i in range(2016, 2022):
    tmp = pd.read_csv(f'./weather_data/{i}.csv', encoding='euc-kr')
    rain_df = rain_df.append(tmp)
rain_df.head()

,지점,지점명,일시,강수량(mm)
0,192,진주,2016-01-05 12:00,0.0
1,192,진주,2016-01-17 18:00,0.8
2,192,진주,2016-01-17 21:00,0.5
3,192,진주,2016-01-18 00:00,0.1
4,192,진주,2016-01-24 06:00,0.5


In [298]:
# 데이터 컬럼에 맞게 전처리
rain_df = rain_df[['일시', '강수량(mm)']].rename(columns={'강수량(mm)':'강수량'}).reset_index(drop=True)
rain_df['일시'] = rain_df['일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M'))
rain_df['일자'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%Y-%m-%d'))
rain_df['시간'] = rain_df['일시'].apply(lambda x : datetime.strftime(x, '%H'))
rain_df['시간'] = rain_df['시간'].astype('int')
rain_df = rain_df[['일자', '시간', '강수량']]
rain_df.head()

,일자,시간,강수량
0,2016-01-05,12,0.0
1,2016-01-17,18,0.8
2,2016-01-17,21,0.5
3,2016-01-18,0,0.1
4,2016-01-24,6,0.5


In [299]:
# 점심, 저녁 5미리 이상만 추출
lunch_rain_df = rain_df[(rain_df['시간'] == 12) & (rain_df['강수량'] >= 5)].reset_index(drop=True)
dinner_rain_df = rain_df[(rain_df['시간'] == 18) & (rain_df['강수량'] >= 5)].reset_index(drop=True)

In [300]:
lunch_rain_df.head()

,일자,시간,강수량
0,2016-02-12,12,9.4
1,2016-05-24,12,8.7
2,2016-06-24,12,10.8
3,2016-09-02,12,5.7
4,2016-09-17,12,10.2


In [301]:
# 점심비, 저녁비 merge
train_df = pd.merge(train_df, lunch_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'점심비'})

train_df = pd.merge(train_df, dinner_rain_df[['일자', '강수량']], left_on='일자', right_on='일자', how='left')
train_df['강수량'] = train_df['강수량'].fillna(0)
train_df = train_df.rename(columns={'강수량':'저녁비'})

In [302]:
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,중식메뉴,현본사소속재택근무자수,휴일전날,점심비,저녁비
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",0.0,0.0,0.0,0.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",0.0,0.0,0.0,0.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",0.0,0.0,0.0,0.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",0.0,0.0,0.0,0.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",0.0,1.0,0.0,0.0


### 원핫 인코딩(요일) 

In [303]:
onehot_tmp = pd.get_dummies(train_df['요일'])
#train_df = pd.concat([train_df.drop(columns='요일'), onehot_tmp], axis=1)

In [304]:
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,중식메뉴,현본사소속재택근무자수,휴일전날,점심비,저녁비
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...",0.0,0.0,0.0,0.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...",0.0,0.0,0.0,0.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...",0.0,0.0,0.0,0.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...",0.0,0.0,0.0,0.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...",0.0,1.0,0.0,0.0


# 코로나 확진자수(진주, 경남, 전국) 데이터 합치기

In [305]:
train_corona=pd.read_csv('./data/train_corona.csv')
test_corona=pd.read_csv('./data/test_corona.csv')

In [306]:
train_corona.drop(['Unnamed: 0'],axis=1,inplace=True)
test_corona.drop(['Unnamed: 0'],axis=1,inplace=True)

In [307]:
train_corona =train_corona.append(test_corona)
train_corona

,일자,전국 코로나 확진자 수,경남 코로나 확진자 수,진주 코로나 확진자 수
0,2016-02-01,0.0,0.0,0.0
1,2016-02-02,0.0,0.0,0.0
2,2016-02-03,0.0,0.0,0.0
3,2016-02-04,0.0,0.0,0.0
4,2016-02-05,0.0,0.0,0.0
5,2016-02-11,0.0,0.0,0.0
6,2016-02-12,0.0,0.0,0.0
7,2016-02-15,0.0,0.0,0.0
8,2016-02-16,0.0,0.0,0.0
9,2016-02-17,0.0,0.0,0.0


In [308]:
train_df=pd.merge(train_df,train_corona,how='left',on='일자')

# 거리두기 관련 변수 추가

In [309]:
import datetime

train_df['일자']=pd.to_datetime(train_df['일자'])

In [310]:
for i in range(len(train_df)):
    if train_df['일자'][i]<datetime.datetime(2020,8,23):
        train_df.loc[i,'사회적거리두기 1단계']=0
        train_df.loc[i,'사회적거리두기 1.5단계']=0
        train_df.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,8,23)<=train_df['일자'][i]<datetime.datetime(2020,10,13):
        train_df.loc[i,'사회적거리두기 1단계']=0
        train_df.loc[i,'사회적거리두기 1.5단계']=0
        train_df.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2020,10,13)<=train_df['일자'][i]<datetime.datetime(2020,12,8):
        train_df.loc[i,'사회적거리두기 1단계']=1
        train_df.loc[i,'사회적거리두기 1.5단계']=0
        train_df.loc[i,'사회적거리두기 2단계 이상']=0
    elif datetime.datetime(2020,12,8)<=train_df['일자'][i]<datetime.datetime(2021,2,15):
        train_df.loc[i,'사회적거리두기 1단계']=0
        train_df.loc[i,'사회적거리두기 1.5단계']=0
        train_df.loc[i,'사회적거리두기 2단계 이상']=1
    elif datetime.datetime(2021,2,15)<=train_df['일자'][i]:
        train_df.loc[i,'사회적거리두기 1단계']=0
        train_df.loc[i,'사회적거리두기 1.5단계']=1
        train_df.loc[i,'사회적거리두기 2단계 이상']=0

# 거리두기 파생변수 생성

In [311]:
train_df['1단계 누적일수']=0
train_df['1.5단계 누적일수']=0
train_df['2단계 누적일수']=0

In [312]:
for i in range(1,len(train)):
    if train_df['사회적거리두기 1단계'][i]==1:
        if train_df['1단계 누적일수'][i-1]==0:
            train_df['1단계 누적일수'][i]=1
        elif train_df['1단계 누적일수'][i-1]!=0:
            train_df['1단계 누적일수'][i]=train_df['1단계 누적일수'][i-1]+int(str(train_df['일자'][i]-train_df['일자'][i-1])[0:2].rstrip())
    elif train_df['사회적거리두기 1.5단계'][i]==1:
        if train_df['1.5단계 누적일수'][i-1]==0:
            train_df['1.5단계 누적일수'][i]=1
        elif train_df['1.5단계 누적일수'][i-1]!=0:
            train_df['1.5단계 누적일수'][i]=train_df['1.5단계 누적일수'][i-1]+int(str(train_df['일자'][i]-train_df['일자'][i-1])[0:2].rstrip())
    elif train_df['사회적거리두기 2단계 이상'][i]==1:
        if train_df['2단계 누적일수'][i-1]==0:
            train_df['2단계 누적일수'][i]=1
        elif train_df['2단계 누적일수'][i-1]!=0:
            train_df['2단계 누적일수'][i]=train_df['2단계 누적일수'][i-1]+int(str(train_df['일자'][i]-train_df['일자'][i-1])[0:2].rstrip())
    else:
        pass

In [313]:
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,...,저녁비,전국 코로나 확진자 수,경남 코로나 확진자 수,진주 코로나 확진자 수,사회적거리두기 1단계,사회적거리두기 1.5단계,사회적거리두기 2단계 이상,1단계 누적일수,1.5단계 누적일수,2단계 누적일수
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


# 거리두기 가중치 추가

In [314]:
for i in range(len(train)):
    if train_df['일자'][i]<datetime.datetime(2020,8,22):
        train_df.loc[i,'격상 가중치']=0
        train_df.loc[i,'격하 가중치']=0
        train_df.loc[i,'가중치']=0

In [315]:
for i in range(len(train_df)):
    if train_df['일자'][i]<datetime.datetime(2020,8,22):
        train_df.loc[i,'가중치']=0
    else:
        if train_df['사회적거리두기 2단계 이상'][i]==1:
            train_df.loc[i,'가중치']=2
        elif train_df['사회적거리두기 1.5단계'][i]==1:
            train_df.loc[i,'가중치']=1.5
        elif train_df['사회적거리두기 1단계'][i]==1:
            train_df.loc[i,'가중치']=1

train_df['가중치_v2']='.'
train_df['가중치_v2'][0]=0
for i in range(len(train_df)-1):
    if train_df['가중치'][i+1]==train_df['가중치'][i]:
        train_df.loc[i+1,'가중치_v2']=train_df['가중치_v2'][i]
    elif train_df['가중치'][i+1]!=train_df['가중치'][i]:
        train_df.loc[i+1,'가중치_v2']=train_df['가중치'][i+1]-train_df['가중치'][i]

In [316]:
import math

def dist_inc(x):
    if x==0:
        return 0
    elif x>0:
        return abs(x)
    elif x<0:
        pass
def dist_dec(x):
    if x==0:
        return 0
    elif x<0:
        return abs(x)
    elif x>0:
        pass

In [317]:
train_df['격상 가중치']=train_df['가중치_v2'].apply(lambda x: dist_inc(x))
train_df['격하 가중치']=train_df['가중치_v2'].apply(lambda x: dist_dec(x))
train_df['격상 가중치']=train_df['격상 가중치'].fillna(0)
train_df['격하 가중치']=train_df['격하 가중치'].fillna(0)

train_df.drop(['가중치','가중치_v2'],axis=1,inplace=True)

In [318]:
train_df.tail()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,...,경남 코로나 확진자 수,진주 코로나 확진자 수,사회적거리두기 1단계,사회적거리두기 1.5단계,사회적거리두기 2단계 이상,1단계 누적일수,1.5단계 누적일수,2단계 누적일수,격상 가중치,격하 가중치
1250,704,2973,174,125,NaN,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,월,2021-04-05,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,NaN,...,14.0,1.0,0.0,1.0,0.0,0,50,0,0.0,0.5
1251,636,2973,170,76,NaN,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,화,2021-04-06,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,NaN,...,12.0,1.0,0.0,1.0,0.0,0,51,0,0.0,0.5
1252,1,2973,214,96,NaN,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,수,2021-04-07,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,NaN,...,7.0,3.0,0.0,1.0,0.0,0,52,0,0.0,0.5
1253,509,2973,238,105,NaN,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,목,2021-04-08,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,NaN,...,25.0,3.0,0.0,1.0,0.0,0,53,0,0.0,0.5
1254,0,2973,268,259,NaN,흑미밥 맑은버섯국 매운사태조림 춘권*타르타르D 열무나물무침 포기김치,금,2021-04-09,모닝롤/토마토샌드 우유/주스 계란후라이 채소죽/흑미밥 대구지리 애호박나물볶음 양상추...,NaN,...,11.0,0.0,0.0,1.0,0.0,0,54,0,0.0,0.5


# 온도 변수 추가

In [319]:
temp=pd.read_csv('./weather_data/temperature.csv',encoding='euckr')

In [320]:
temp=temp.rename(columns={'날짜':'일자','평균기온(℃)':'평균기온','최저기온(℃)':'최저기온','최고기온(℃)':'최고기온'})
temp['일자']=pd.to_datetime(temp['일자'])
train_df=pd.merge(train_df,temp,how='left',on='일자')
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,...,사회적거리두기 2단계 이상,1단계 누적일수,1.5단계 누적일수,2단계 누적일수,격상 가중치,격하 가중치,지점,평균기온,최저기온,최고기온
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,...,0.0,0,0,0,0.0,0.0,192,-0.6,-4.1,5.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,...,0.0,0,0,0,0.0,0.0,192,-2.3,-9.1,6.2
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,...,0.0,0,0,0,0.0,0.0,192,-1.7,-9.3,7.7
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,...,0.0,0,0,0,0.0,0.0,192,-0.2,-7.7,7.7
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,...,0.0,0,0,0,0.0,0.0,192,1.3,-6.5,9.6


# 습도 변수 추가

In [321]:
rh_data=pd.read_csv('./weather_data/rh_data.csv',encoding='euckr')

In [322]:
rh_data=rh_data.rename(columns={'일시':'일자','평균습도(%rh)':'평균습도','최저습도(%rh)':'최저습도'})
rh_data['일자']=pd.to_datetime(rh_data['일자'])
train_df=pd.merge(train_df,rh_data,how='left',on='일자')
train_df.drop(['\t\t지점번호','지점명','지점'],axis=1,inplace=True)

In [323]:
train_df.head()

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,...,1단계 누적일수,1.5단계 누적일수,2단계 누적일수,격상 가중치,격하 가중치,평균기온,최저기온,최고기온,평균습도,최저습도
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",월,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,...,0,0,0,0.0,0.0,-0.6,-4.1,5.0,43.9,17.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",화,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,...,0,0,0,0.0,0.0,-2.3,-9.1,6.2,47.5,13.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",수,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,...,0,0,0,0.0,0.0,-1.7,-9.3,7.7,57.1,20.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",목,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,...,0,0,0,0.0,0.0,-0.2,-7.7,7.7,53.4,20.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",금,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,...,0,0,0,0.0,0.0,1.3,-6.5,9.6,44.4,10.0


# 파생 변수 추가

In [324]:
train_df['월'] = pd.DatetimeIndex(train_df['일자']).month

train_df['일'] = pd.DatetimeIndex(train_df['일자']).day

weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train_df['요일'] = train_df['요일'].map(weekday)

train_df['식사가능자수'] = train_df['본사정원수'] - train_df['본사휴가자수'] - train_df['현본사소속재택근무자수']

# 메뉴 분류 추가

In [325]:
menu = pd.read_excel("data/menu_category_ingredient.xlsx")

In [326]:
menu['분류'].unique()

array(['볶음류', '무침류', '조림류', '찜류', '스프류', '밥류', '튀김류', '찌개류', '과일류',
       '샐러드류', '소스류', '면류', '빵과자류', '구이류', '음료류', '국탕류', '회류', '떡류',
       '비빔밥볶음밥류', '죽류', '묵류', '덮밥국밥류', '나물류', '전류', '채소류', '만두류', '김치류',
       '디저트류', '부침류', '유제품류', '해조류', '장아찌류', '견과류', '덥밥국밥류'], dtype=object)

In [327]:
menu['재료1'].unique()

array(['육류', '채소류', '곡류', '어패류', '과일류', '양념 및 장류', '난류', '두류', '김치류',
       '해조류', '유제품류', '만두류'], dtype=object)

In [328]:
menu['재료2'].unique()

array(['채소류', nan, '해조류', '곡류', '김치류', '양념 및 장류', '과일류', '어패류', '난류',
       '유제품류', '육류', '두류', '견과류'], dtype=object)

In [329]:
a = menu['분류'].unique().tolist()
b = menu['재료1'].unique().tolist()
c = menu['재료2'].unique().tolist()
menu_list = list(set(a+b+c))[1:]

In [330]:
len(menu_list)

40

In [331]:
# Basic text preprocessing
def split_process(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        elif '*' in i:
            x_.extend(i.split('*'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_

In [332]:
train_df['조식메뉴_split'] = train_df['조식메뉴'].apply(lambda x: split_process(x)).to_list()
train_df['중식메뉴_split'] = train_df['중식메뉴'].apply(lambda x: split_process(x)).to_list()
train_df['석식메뉴_split'] = train_df['석식메뉴'].apply(lambda x: split_process(x)).to_list()

In [333]:
lunch_lst = []
lunch_count = []
pref = 0
for i in tqdm(range(1205)):
    try:
        if train_df['중식계'][i] >= 880:
            pref = 1
        else:
            pref = 0

        lunch_lst.append(train_df['중식메뉴_split'][i])
        lunch_count.append(pref)
    except:
        pass
print('done')

100%|███████████████████████████████████| 1205/1205 [00:00<00:00, 13101.80it/s]


done


In [334]:
dinner_lst = []
dinner_count = []
pref_d = 0
for i in tqdm(range(1205)):
    try:
        if train_df['석식계'][i] > 476:
            pref_d = 1
        else:
            pref_d = 0

        dinner_lst.append(train_df['석식메뉴_split'][i])
        dinner_count.append(pref_d)
    except:
        pass
print('done')

100%|███████████████████████████████████| 1205/1205 [00:00<00:00, 12961.15it/s]


done


In [335]:
lunch_df = pd.DataFrame({'중식메뉴':lunch_lst})
dinner_df = pd.DataFrame({'석식메뉴':dinner_lst})

In [336]:
lunch_df

,중식메뉴
0,"[포기김치, 요구르트, 계란찜, 오징어찌개, 쇠불고기, 잡곡밥, 청포묵무침, 쌀밥]"
1,"[요구르트, 배추겉절이, 김치찌개, 가자미튀김, 잡곡밥, 마늘쫑무침, 쌀밥, 모둠소..."
2,"[견과류조림, 포기김치, 쫄면야채무침, 팽이장국, 요구르트, 카레덮밥, 치킨핑거]"
3,"[포기김치, 요구르트, 시금치나물, 주꾸미볶음, 쇠고기무국, 잡곡밥, 부추전, 쌀밥]"
4,"[돈육씨앗강정, 우엉잡채, 포기김치, 요구르트, 청경채무침, 떡국, 잡곡밥, 쌀밥]"
5,"[도토리묵무침, 포기김치, 요구르트, 훈제오리구이, 양파절임, 잡곡밥, 쌈무, 쌀밥..."
6,"[포기김치, 요구르트, 옥수수전, 유채나물, 잡곡밥, 돈육굴소스볶음, 꽃게탕, 쌀밥]"
7,"[포기김치, 닭감자조림, 시금치국, 요구르트, 양념장, 잡곡밥, 콩나물무침, 연두부..."
8,"[포기김치, 요구르트, 오징어숙회무침, 취나물, 탕수어, 쇠고기무국, 잡곡밥, 쌀밥]"
9,"[쇠고기장조림, 치커리무침, 포기김치, 요구르트, 통도라지구이, 잡곡밥, 냉이된장국..."


In [337]:
food = list(menu['메뉴'])
food

['대패삼겹숙주볶음',
 '봄동나물',
 '콩나물간장조림',
 '닭찜',
 '아삭고추무침',
 '스프',
 '쌀밥',
 '수제돈가스',
 '수제어묵볶음',
 '차돌박이된장찌개',
 '문어꽈리고추조림',
 '파프리카해초무침',
 '바나나',
 '참치야채비빔밥',
 '브로컬리두부무침',
 '갈치감자조림',
 '양상추샐러드',
 '오미지D',
 '김치국수',
 '생크림와플',
 '김계란말이',
 '푸실리샐러드',
 '커피',
 '버섯육개장',
 '김주먹밥',
 '페스츄리',
 '크래미해초무침',
 '오징어&브로컬리숙회',
 '새우까스',
 '소스',
 '버섯국',
 '매운콩나물무침',
 '사과오이냉국',
 '냉이바지락국',
 '나쵸칩',
 '깐풍연근',
 '국물떡볶이',
 '오리훈제볶음밥',
 '닭죽',
 '새송이버섯볶음',
 '고추간장지',
 '훈제오리떡볶음',
 '열무김치국수',
 '호박잎찌개',
 '시금치프리타타',
 '두부스테이크',
 '포도',
 '볼어묵곤약볶음',
 '비엔나피망볶음',
 '참치주먹밥',
 '모듬소세지버섯구이',
 '야채볶음밥',
 '야채볶음밥',
 '짜장소스',
 '콩나물무침',
 '도토리묵',
 '장',
 '연두부찌개',
 '콘스프',
 '물냉면',
 '돈육볶음',
 '순대국밥',
 '버블샐러드',
 'D오리엔탈D',
 '봄동숙',
 '간장양념',
 '키위드레싱샐러드',
 '쌈장',
 '쌈장',
 '쌈장',
 '쌈장',
 '오이맛살초무침',
 '모듬장조림',
 '해물동그랑땡',
 '삼겹살수육',
 '풋고추',
 '레몬유린기',
 '(New)칠리가지탕수',
 '참나물무침',
 '유부채소겨자냉채',
 '등심찹쌀꿔바로우',
 '옹심이국',
 '생선까스',
 '생선까스',
 '칠리소스',
 '망고드레싱',
 '망고드레싱',
 '조랭이떡국',
 '감자조림',
 '카레감자채볶음',
 '주꾸미브로콜리숙회',
 '야채쫄면',
 '사과오이생채',
 '분홍소세지구이',
 '케찹',
 '비빔메밀국수',
 '버섯매운탕',
 '당면계란만두',
 '냉이콩나물국'

In [338]:
dictionary = {string : i for i,string in enumerate(menu_list)}
dictionary

{'전류': 0,
 '볶음류': 1,
 '만두류': 2,
 '유제품류': 3,
 '조림류': 4,
 '구이류': 5,
 '밥류': 6,
 '양념 및 장류': 7,
 '스프류': 8,
 '소스류': 9,
 '두류': 10,
 '디저트류': 11,
 '찌개류': 12,
 '채소류': 13,
 '묵류': 14,
 '회류': 15,
 '비빔밥볶음밥류': 16,
 '덥밥국밥류': 17,
 '김치류': 18,
 '국탕류': 19,
 '튀김류': 20,
 '빵과자류': 21,
 '장아찌류': 22,
 '나물류': 23,
 '해조류': 24,
 '덮밥국밥류': 25,
 '어패류': 26,
 '난류': 27,
 '찜류': 28,
 '떡류': 29,
 '면류': 30,
 '무침류': 31,
 '샐러드류': 32,
 '부침류': 33,
 '음료류': 34,
 '과일류': 35,
 '견과류': 36,
 '곡류': 37,
 '육류': 38,
 '죽류': 39}

In [339]:
menu[menu['메뉴']==lunch_df.iloc[0][0][1]].iloc[0][1:]

분류     유제품류
재료1    유제품류
재료2     NaN
Name: 1956, dtype: object

In [340]:
#train data
result_lunch = []
result_dinner = []

for k in range(len(lunch_df)): #len(lunch_df)
    trigger = np.zeros(40, dtype='int')
    for i in range(len(lunch_df.iloc[k][0])):
        if lunch_df.iloc[k][0][i] in food:  #메뉴 하나 하나
            #for j,category in enumerate(menu_list):
            for x in range(0,3):
                try:
                    loc = dictionary[menu[menu['메뉴']==lunch_df.iloc[k][0][i]].iloc[0][1:][x]]
                    trigger[loc] += 1
                except:
                    continue
    result_lunch.append(trigger)
    
for k in range(len(dinner_df)): #len(lunch_df)
    trigger = np.zeros(40, dtype='int')
    for i in range(len(dinner_df.iloc[k][0])):
        if dinner_df.iloc[k][0][i] in food:  #메뉴 하나 하나
            #for j,category in enumerate(menu_list):
            for x in range(0,3):
                try:
                    loc = dictionary[menu[menu['메뉴']==dinner_df.iloc[k][0][i]].iloc[0][1:][x]]
                    trigger[loc] += 1
                except:
                    continue
    result_dinner.append(trigger)

In [341]:
lunch_sort = pd.DataFrame(result_lunch, columns=menu_list)
dinner_sort = pd.DataFrame(result_dinner, columns=menu_list)

In [342]:
result_lunch

[array([0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0]),
 array([0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0]),
 array([0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0]),
 array([1, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0]),
 array([0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 1, 0]),
 array([0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 1, 0]),
 array([1, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0]),
 array([0, 0, 0, 2, 1, 0, 2, 1, 0,

In [343]:
lunch_sort

,전류,볶음류,만두류,유제품류,조림류,구이류,밥류,양념 및 장류,스프류,소스류,...,면류,무침류,샐러드류,부침류,음료류,과일류,견과류,곡류,육류,죽류
0,0,0,0,2,0,1,2,0,0,0,...,0,1,0,0,0,0,0,2,1,0
1,0,0,0,2,0,1,2,0,0,0,...,0,1,0,0,0,0,0,2,1,0
2,0,0,0,2,1,0,0,1,0,0,...,0,1,0,0,0,0,0,2,1,0
3,1,1,0,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,2,1,0
4,0,1,0,2,0,0,2,0,0,0,...,0,1,0,0,0,0,0,3,1,0
5,0,0,0,2,0,1,2,0,0,0,...,0,3,0,0,0,0,0,2,1,0
6,1,1,0,2,0,0,2,0,0,0,...,0,0,0,0,0,0,0,3,1,0
7,0,0,0,2,1,0,2,1,0,1,...,0,1,0,0,0,0,0,2,1,0
8,0,0,0,2,0,0,2,0,0,0,...,0,1,0,0,0,0,0,2,1,0
9,0,0,0,2,1,1,2,0,0,0,...,0,1,0,0,0,0,0,2,1,0


In [344]:
lunch_menu_train = lunch_sort.div(lunch_sort.sum(axis=1),axis=0)
dinner_menu_train = dinner_sort.div(dinner_sort.sum(axis=1),axis=0)

In [345]:
train_lunch_final=pd.concat([train_df,lunch_menu_train],axis=1)
train_dinner_final=pd.concat([train_df,dinner_menu_train],axis=1)

In [346]:
train_lunch_final

,본사시간외근무명령서승인건수,본사정원수,본사출장자수,본사휴가자수,석식계,석식메뉴,요일,일자,조식메뉴,중식계,...,면류,무침류,샐러드류,부침류,음료류,과일류,견과류,곡류,육류,죽류
0,238,2601,150,50,331.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,1039.0,...,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.062500,0.0
1,319,2601,173,50,560.0,"콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",2,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,867.0,...,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.062500,0.0
2,111,2601,180,56,573.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",3,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,1017.0,...,0.0000,0.071429,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.071429,0.0
3,355,2601,220,104,525.0,"미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",4,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...",978.0,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.062500,0.0
4,34,2601,181,278,330.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",5,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,925.0,...,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.187500,0.062500,0.0
5,417,2601,143,383,550.0,"참치회덮밥 (쌀,현미흑미:국내산) 맑은국 군만두 과일샐러드 락교 포기김치 (...",4,2016-02-11,팬케익/찐빵 우유/두유/주스 계란후라이 견과류죽/쌀밥 (쌀:국내산) 감자찌개 ...,1045.0,...,0.0000,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.055556,0.0
6,93,2601,156,389,598.0,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치콩나물국 미니함박 어묵볶음 물파래무침 ...",5,2016-02-12,모닝롤/야채샌드 우유/두유/주스 계란후라이 고구마죽/쌀밥 (쌀:국내산) 봄동된...,909.0,...,0.0000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.187500,0.062500,0.0
7,482,2601,204,87,672.0,"쌀밥/잡곡밥 (쌀:국내산) 홍합미역국 등갈비김치찜 (돼지고기,김치:국내산) 임연수...",1,2016-02-15,모닝롤/치즈프레즐 우유/두유/주스 계란후라이 잣죽/쌀밥 (쌀:국내산) 민물새우...,1268.0,...,0.0000,0.055556,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.055556,0.0
8,526,2601,236,72,523.0,쌀밥/잡곡밥 (쌀:국내산) 된장찌개 쇠불고기 (쇠고기:호주산) 해파리겨자채 봄동...,2,2016-02-16,모닝롤/마늘빵 우유/두유/주스 계란후라이 단호박죽/쌀밥 (쌀:국내산) 어묵국 ...,1014.0,...,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.062500,0.0
9,23,2601,250,78,588.0,볶음밥*자장소스 (쌀:국내산) 맑은국 새우또띠아 쨔샤이무침 요플레 포기김치 ...,3,2016-02-17,모닝롤/참치샌드 우유/두유/주스 계란후라이 흑임자죽/쌀밥 (쌀:국내산) 북어계...,916.0,...,0.0000,0.062500,0.0,0.0,0.0,0.000000,0.000000,0.125000,0.062500,0.0


In [347]:
train_lunch_final.drop(['일자','조식메뉴','중식메뉴','석식메뉴'],axis=1,inplace=True)
train_dinner_final.drop(['일자','조식메뉴','중식메뉴','석식메뉴'],axis=1,inplace=True)

In [348]:
train_lunch_final = train_lunch_final.fillna(0)
train_dinner_final = train_dinner_final.fillna(0)

In [349]:
train_lunch_final.columns

Index(['본사시간외근무명령서승인건수', '본사정원수', '본사출장자수', '본사휴가자수', '석식계', '요일', '중식계',
       '현본사소속재택근무자수', '휴일전날', '점심비', '저녁비', '전국 코로나 확진자 수', '경남 코로나 확진자 수',
       '진주 코로나 확진자 수', '사회적거리두기 1단계', '사회적거리두기 1.5단계', '사회적거리두기 2단계 이상',
       '1단계 누적일수', '1.5단계 누적일수', '2단계 누적일수', '격상 가중치', '격하 가중치', '평균기온',
       '최저기온', '최고기온', '평균습도', '최저습도', '월', '일', '식사가능자수', '조식메뉴_split',
       '중식메뉴_split', '석식메뉴_split', '전류', '볶음류', '만두류', '유제품류', '조림류', '구이류',
       '밥류', '양념 및 장류', '스프류', '소스류', '두류', '디저트류', '찌개류', '채소류', '묵류', '회류',
       '비빔밥볶음밥류', '덥밥국밥류', '김치류', '국탕류', '튀김류', '빵과자류', '장아찌류', '나물류', '해조류',
       '덮밥국밥류', '어패류', '난류', '찜류', '떡류', '면류', '무침류', '샐러드류', '부침류', '음료류',
       '과일류', '견과류', '곡류', '육류', '죽류'],
      dtype='object')

In [350]:
len(train_lunch_final.columns)

73

# 모델링 전 데이터프레임 및 변수정리

In [351]:
test_lunch_final=train_lunch_final[1205:].reset_index(drop=True)
train_lunch_final=train_lunch_final[:1205]
test_dinner_final=train_dinner_final[1205:].reset_index(drop=True)
train_dinner_final=train_dinner_final[:1205]

In [352]:
train_lunch_final.drop(['최저기온','최저습도', '조식메뉴_split','중식메뉴_split', '석식메뉴_split'],axis=1,inplace=True)
test_lunch_final.drop(['최저기온','최저습도', '조식메뉴_split','중식메뉴_split', '석식메뉴_split'],axis=1,inplace=True)
train_dinner_final.drop(['최저기온','최저습도', '조식메뉴_split','중식메뉴_split', '석식메뉴_split'],axis=1,inplace=True)
test_dinner_final.drop(['최저기온','최저습도', '조식메뉴_split','중식메뉴_split', '석식메뉴_split'],axis=1,inplace=True)

In [353]:
y1_train = train_lunch_final['중식계']
y2_train = train_dinner_final['석식계']

In [354]:
train_lunch_final.drop(['중식계','석식계','평균기온'],axis=1,inplace=True)
test_lunch_final.drop(['중식계','석식계','평균기온'],axis=1,inplace=True)
train_dinner_final.drop(['중식계','석식계','최고기온'],axis=1,inplace=True)
test_dinner_final.drop(['중식계','석식계','최고기온'],axis=1,inplace=True)

# 결측값 확인

In [355]:
for i in range(len(train_lunch_final.columns)):
    print(train_lunch_final.columns[i],' 결측값 : ',len(train_lunch_final[train_lunch_final[train_lunch_final.columns[i]].isna()==True]))

본사시간외근무명령서승인건수  결측값 :  0
본사정원수  결측값 :  0
본사출장자수  결측값 :  0
본사휴가자수  결측값 :  0
요일  결측값 :  0
현본사소속재택근무자수  결측값 :  0
휴일전날  결측값 :  0
점심비  결측값 :  0
저녁비  결측값 :  0
전국 코로나 확진자 수  결측값 :  0
경남 코로나 확진자 수  결측값 :  0
진주 코로나 확진자 수  결측값 :  0
사회적거리두기 1단계  결측값 :  0
사회적거리두기 1.5단계  결측값 :  0
사회적거리두기 2단계 이상  결측값 :  0
1단계 누적일수  결측값 :  0
1.5단계 누적일수  결측값 :  0
2단계 누적일수  결측값 :  0
격상 가중치  결측값 :  0
격하 가중치  결측값 :  0
최고기온  결측값 :  0
평균습도  결측값 :  0
월  결측값 :  0
일  결측값 :  0
식사가능자수  결측값 :  0
전류  결측값 :  0
볶음류  결측값 :  0
만두류  결측값 :  0
유제품류  결측값 :  0
조림류  결측값 :  0
구이류  결측값 :  0
밥류  결측값 :  0
양념 및 장류  결측값 :  0
스프류  결측값 :  0
소스류  결측값 :  0
두류  결측값 :  0
디저트류  결측값 :  0
찌개류  결측값 :  0
채소류  결측값 :  0
묵류  결측값 :  0
회류  결측값 :  0
비빔밥볶음밥류  결측값 :  0
덥밥국밥류  결측값 :  0
김치류  결측값 :  0
국탕류  결측값 :  0
튀김류  결측값 :  0
빵과자류  결측값 :  0
장아찌류  결측값 :  0
나물류  결측값 :  0
해조류  결측값 :  0
덮밥국밥류  결측값 :  0
어패류  결측값 :  0
난류  결측값 :  0
찜류  결측값 :  0
떡류  결측값 :  0
면류  결측값 :  0
무침류  결측값 :  0
샐러드류  결측값 :  0
부침류  결측값 :  0
음료류  결측값 :  0
과일류  결측값 :  0
견과류  결측값 :  0
곡류  결측값 :  0
육류  

In [356]:
for i in range(len(train_dinner_final.columns)):
    print(train_dinner_final.columns[i],' 결측값 : ',len(train_dinner_final[train_dinner_final[train_dinner_final.columns[i]].isna()==True]))

본사시간외근무명령서승인건수  결측값 :  0
본사정원수  결측값 :  0
본사출장자수  결측값 :  0
본사휴가자수  결측값 :  0
요일  결측값 :  0
현본사소속재택근무자수  결측값 :  0
휴일전날  결측값 :  0
점심비  결측값 :  0
저녁비  결측값 :  0
전국 코로나 확진자 수  결측값 :  0
경남 코로나 확진자 수  결측값 :  0
진주 코로나 확진자 수  결측값 :  0
사회적거리두기 1단계  결측값 :  0
사회적거리두기 1.5단계  결측값 :  0
사회적거리두기 2단계 이상  결측값 :  0
1단계 누적일수  결측값 :  0
1.5단계 누적일수  결측값 :  0
2단계 누적일수  결측값 :  0
격상 가중치  결측값 :  0
격하 가중치  결측값 :  0
평균기온  결측값 :  0
평균습도  결측값 :  0
월  결측값 :  0
일  결측값 :  0
식사가능자수  결측값 :  0
전류  결측값 :  0
볶음류  결측값 :  0
만두류  결측값 :  0
유제품류  결측값 :  0
조림류  결측값 :  0
구이류  결측값 :  0
밥류  결측값 :  0
양념 및 장류  결측값 :  0
스프류  결측값 :  0
소스류  결측값 :  0
두류  결측값 :  0
디저트류  결측값 :  0
찌개류  결측값 :  0
채소류  결측값 :  0
묵류  결측값 :  0
회류  결측값 :  0
비빔밥볶음밥류  결측값 :  0
덥밥국밥류  결측값 :  0
김치류  결측값 :  0
국탕류  결측값 :  0
튀김류  결측값 :  0
빵과자류  결측값 :  0
장아찌류  결측값 :  0
나물류  결측값 :  0
해조류  결측값 :  0
덮밥국밥류  결측값 :  0
어패류  결측값 :  0
난류  결측값 :  0
찜류  결측값 :  0
떡류  결측값 :  0
면류  결측값 :  0
무침류  결측값 :  0
샐러드류  결측값 :  0
부침류  결측값 :  0
음료류  결측값 :  0
과일류  결측값 :  0
견과류  결측값 :  0
곡류  결측값 :  0
육류  

# MAE 사용자 정의 함수

In [357]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [358]:
def get_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid=params, scoring='neg_mean_absolute_error', cv=5)
    grid_model.fit(train_lunch_final, y1_train)
    mae = abs(-1* grid_model.best_score_)
    print('최적 MAE 값:', np.round(mae, 4))
    print('최적 파라미터:', grid_model.best_params_)
    
    return grid_model.best_estimator_

In [359]:
def get_best_params2(model, params):
    grid_model = GridSearchCV(model, param_grid=params, scoring='neg_mean_absolute_error', cv=5)
    grid_model.fit(train_dinner_final, y2_train)
    mae = abs(-1* grid_model.best_score_)
    print('최적 MAE 값:', np.round(mae, 4))
    print('최적 파라미터:', grid_model.best_params_)
    
    return grid_model.best_estimator_

# 중식계 모델링

In [360]:
final_lunch_gbmmodel=GradientBoostingRegressor()

In [361]:
gbm_params1 = {'n_estimators':[700]}

best_gbm1 = get_best_params(final_lunch_gbmmodel, gbm_params1)

최적 MAE 값: 79.9377
최적 파라미터: {'n_estimators': 700}


In [362]:
best_gbm1

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=700,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [363]:
final_lunch_gbmmodel=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=700, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [364]:
final_lunch_gbmmodel.fit(train_lunch_final, y1_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=700,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [365]:
pred_lunch_fin = final_lunch_gbmmodel.predict(test_lunch_final)

# 석식계 모델링

In [366]:
final_dinner_gbmmodel=GradientBoostingRegressor()

In [369]:
gbm_params2 = {'n_estimators':[700]}

best_gbm2 = get_best_params2(final_dinner_gbmmodel, gbm_params2)

최적 MAE 값: 62.0692
최적 파라미터: {'n_estimators': 700}


In [370]:
best_gbm2

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=700,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [371]:
final_dinner_gbmmodel=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=700, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [372]:
final_dinner_gbmmodel.fit(train_dinner_final, y2_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=700,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [373]:
pred_dinner_fin = final_dinner_gbmmodel.predict(test_dinner_final)

# 예측값 저장

In [374]:
submission['중식계'] = pred_lunch_fin
submission['석식계'] = pred_dinner_fin

In [375]:
submission.to_csv('submission_final_code.csv', index=False)